# Spam Soaker 2000

I have affectionately dubbed this spam filter, the Spam Soaker 2000. What it lacks in real world applicability, it more than makes up for with it's super cool name.


## Building a Spam Filter with Naive Bayes

In this project, we will be building a spam filter using the multinomial Naive Bayes algorithm.

The data was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the <a href="https://archive.ics.uci.edu/ml/datasets/sms+spam+collection">UCI Machine Learning Repository</a>. The data collection process is described in more details on <a href="http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition">this page</a>, where you can also find some of the authors' papers.

Let's begin with reading in this dataset and getting familiar with it:

In [150]:
import pandas as pd
data = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

In [151]:
data

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"
...,...,...
5567,spam,"This is the 2nd time we have tried 2 contact u. U have won the £750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate."
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other suggestions?"
5570,ham,The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free


In [152]:
data['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

We have 5572 text messages, which is actually 2 less than expected, but this shouldn't make too much difference. We can also see that around 86.6% of messages in the data have been clasified as non-spam (referred to as 'ham' in the data), while 13.4% have been classified as spam- this seems like a reasonable proportion.

Now, let's split our data into training and testing. We will go for a training - testing split of 80 - 20, so 4458 messages will be in the training data, leaving 1114 in the testing data

In [153]:
# Note we first randomize the dataframe, then split it to avoid overlap
training_data = data.sample(frac=1, random_state=1)[:4458]
testing_data = data.sample(frac=1, random_state=1)[4458:]

As a little sanity check, let's verify that the percentages of spam and non-spam:

In [154]:
training_data['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [155]:
testing_data['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

Everything looking nice and random. Hooray!

## Quick and Dirty Naive Bayes Derivation

Before we begin training our model, let's recap the algorithm we will be using- multinomial Naive Bayes. Firstly, we will define events:

$Spam = \text {event that message is spam}$
<br>
$w_i = \text {event that message contains word } w_i$

We now need to use the training data to estimate some probabilities of some events, which we require to feed into Bayes theorem.

$P(Spam) = \text{proportion of spam messages in training data}$

$P(Ham) = \text{proportion of ham messages in training data}$

$P(w_i\ |\ Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}$

$P(w_i\ |\ Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}$
<br>
<br>
where:
<br>
<br>
$N_{w_i|Spam} = \text{the number of times the word } w_i \text{ occurs in spam messages}$
$N_{w_i|Spam^C} = \text{the number of times the word } w_i \text{ occurs in non-spam messages}$
<br>
<br>
$N_{Spam} = \text{total number of words in spam messages}$
$N_{Spam^C} = \text{total number of words in non-spam messages}$
<br>
<br>
$N_{Vocabulary} = \text{total number of words in the vocabulary}$

$\alpha = 1 \ \ \ \ (\alpha \text{ is a smoothing parameter})$
<br>
<br>
<br>
<br>
<br>
Ignoring the $\alpha$ terms in the last two expressions, you can see we qutie simply just estimate these based on their relative proportions in the training data

(note: the terms containing $\alpha$ are used for **additive smoothing**- they prevent these probabilities being 0 when using a word which does not occur in the training data)




Now, for a particular message we want to classify, containing words $w_1,w_2, ..., w_n$, we need to calculate the following two probabilities:

$P(Spam\ |\ w_1,w_2, ..., w_n)$

$P(Ham\ |\ w_1,w_2, ..., w_n)$

Then, depending on which probability is larger, our model will classify this message accordingly. Let's now calculate these probabilities.

According to Bayes' Theorem, we have:


$P(Spam\ |\ w_1,w_2, ..., w_n) = \frac{P(Spam) \cdot P(w_1,w_2, ..., w_n\ |\  Spam)}{P(w_1,w_2, ..., w_n)}$

$P(Ham\ |\ w_1,w_2, ..., w_n) = \frac{P(Ham) \cdot P(w_1,w_2, ..., w_n\ |\  Ham)}{P(w_1,w_2, ..., w_n)}$

However, recall we only care about which of these two values is larger, so for efficiency, we can ignore the divisor $w_1,w_2, ..., w_n$, so let's rewrite this as follows:

$P(Spam\ |\ w_1,w_2, ..., w_n) \propto P(Spam) \cdot P(w_1,w_2, ..., w_n\ |\  Spam)$

$P(Ham\ |\ w_1,w_2, ..., w_n) \propto P(Ham) \cdot P(w_1,w_2, ..., w_n\ |\  Ham)$

Now, in order to calculate:

$P(w_1,w_2, ..., w_n\ |\  Spam)$

$P(w_1,w_2, ..., w_n\ |\  Ham)$

We make the (wildly unrealistic) assumption, that the events $w_i$ have conditional independence, and thus:

$P(w_1,w_2, ..., w_n\ |\  Spam) =  \prod_{i=1}^{n}P(w_i\ |\ Spam)$

$P(w_1,w_2, ..., w_n\ |\  Ham) =  \prod_{i=1}^{n}P(w_i\ |\ Ham)$

which gives us:

$P(Spam\ |\ w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i\ |\ Spam)$

$P(Ham\ |\ w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i\ |\ Ham)$

So, it's clear under the assumption of conditional independence, the probabilities we estiamte using our training data are enough to classify new messages. 

## Data Cleaning

Let's do some quick data cleaning. Our first step is to remove all punctuation, and make all words lower case.

In [156]:
training_data['SMS'] = training_data['SMS'].str.replace('\W',' ').str.lower()

Now we need to collect the unique set of words which occur in the training data, otherwise known as the **vocabulary**.

In [157]:
vocabulary = []

# make word list from each SMS
training_data['SMS'] = training_data['SMS'].str.split()

# aggregate all word lists into one
for sms in training_data['SMS']:
    vocabulary += sms
    
# remove dupes
vocabulary = list(set(vocabulary))
    

We now use some clever logic to find how many occurrences of each word occur in each SMS

In [158]:
word_counts_per_sms = {unique_word: [0] * len(training_data['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_data['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [159]:
word_counts_df = pd.DataFrame(word_counts_per_sms)

In [160]:
training_data = training_data.reset_index(drop=True)

In [161]:
training_data = pd.concat([training_data, word_counts_df], axis=1)

## Training the Model

Now the fun begins, let's create our spam filter!

In [162]:
# splitting the dataframe into spam and ham
spam = training_data[training_data['Label'] == 'spam']
ham = training_data[training_data['Label'] == 'ham']

In [163]:
# calculating the probability of spam or ham (our prior probabilities for Bayes' Theorem)
p_spam = len(spam) / 4458
p_ham = len(ham) / 4458

In [164]:
# calculating the total number of words in spam messages
n_spam = 0

for sms in spam['SMS']:
    len_sms = len(sms)
    n_spam += len_sms

In [165]:
# calculating the total number of words in ham messages
n_ham = 0

for sms in ham['SMS']:
    len_sms = len(sms)
    n_ham += len_sms

In [166]:
# calculating number of unique words in training data
n_vocabulary = len(vocabulary)

In [167]:
alpha = 1

In [168]:
# initialize two dictionaries to store probabilities
spam_dict = {word : 0 for word in vocabulary}
ham_dict = {word : 0 for word in vocabulary}

In [169]:
# create list of all words in spam and ham messages

spam_words = []
for sms in spam['SMS']:
    spam_words += sms
    
ham_words = []
for sms in ham['SMS']:
    ham_words += sms

We now have everything we need to calculate $P(w_i\ |\ Spam)$ for all words in the vocabulary

In [171]:
for unique_word in vocabulary:
    # calculating how manay times each word in vocabulary occurs in spam and ham messages
    spam_count = 0
    ham_count = 0
    for word in spam_words:
        if word == unique_word:
            spam_count += 1
    
    for word in ham_words:
        if word == unique_word:
            ham_count += 1
            
    # calculate probability that word is spam / ham given it contains the word
    p_word_spam = (spam_count + alpha) / (n_spam + alpha * n_vocabulary)
    p_word_ham = (ham_count + alpha) / (n_ham + alpha * n_vocabulary)
    
    # finally update dictionaries with their respective probabilities
    spam_dict[unique_word] = p_word_spam
    ham_dict[unique_word] = p_word_ham
            

We now define our spam filter function:

In [172]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    # we need to calculate the probabilities that the message is spam given message
    p_spam_given_message = p_spam
    for word in message:
        if word in vocabulary:
            prob = spam_dict[word]
        else:
            prob = alpha / (n_spam + alpha * n_vocabulary)
            
        p_spam_given_message *= prob
     
    p_ham_given_message = p_ham
    for word in message:
        if word in vocabulary:
            prob = ham_dict[word]
        else:
            prob = alpha / (n_ham + alpha * n_vocabulary)
            
        p_ham_given_message *= prob
    
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

So, we have built our spam classifier! Let's try it on a few simple messages:

In [173]:
classify('Sounds good, Alex, then see u there')

P(Spam|message): 1.2186187832944059e-25
P(Ham|message): 8.604237681688223e-21
Label: Ham


In [174]:
classify('YOU WIN THE PRIZE MONEY JACKPOT! CALL 14')

P(Spam|message): 3.2592569170008664e-24
P(Ham|message): 4.578068659141562e-28
Label: Spam


With obvious spam and non-spam, the classifier seems to be working in the way we would expect. Let's properly evaluate model performance using test data now. We just need to update our function to actual return something first, rather than print

In [175]:

def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    # we need to calculate the probabilities that the message is spam given message
    p_spam_given_message = p_spam
    for word in message:
        if word in vocabulary:
            prob = spam_dict[word]
        else:
            prob = alpha / (n_spam + alpha * n_vocabulary)
            
        p_spam_given_message *= prob
     
    p_ham_given_message = p_ham
    for word in message:
        if word in vocabulary:
            prob = ham_dict[word]
        else:
            prob = alpha / (n_ham + alpha * n_vocabulary)
            
        p_ham_given_message *= prob

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [176]:
testing_data['predicted'] = testing_data['SMS'].apply(classify_test_set)

In [177]:
testing_data

,Label,SMS,predicted
2131,ham,Later i guess. I needa do mcat study too.,ham
3418,ham,But i haf enuff space got like 4 mb...,ham
3424,spam,Had your mobile 10 mths? Update to latest Orange camera/video phones for FREE. Save £s with Free texts/weekend calls. Text YES for a callback orno to opt out,spam
1538,ham,All sounds good. Fingers . Makes it difficult to type,ham
5393,ham,"All done, all handed in. Don't know if mega shop in asda counts as celebration but thats what i'm doing!",ham
...,...,...,...
905,ham,"We're all getting worried over here, derek and taylor have already assumed the worst",ham
5192,ham,Oh oh... Den muz change plan liao... Go back have to yan jiu again...,ham
3980,ham,CERI U REBEL! SWEET DREAMZ ME LITTLE BUDDY!! C YA 2MORO! WHO NEEDS BLOKES,ham
235,spam,Text & meet someone sexy today. U can find a date or even flirt its up to U. Join 4 just 10p. REPLY with NAME & AGE eg Sam 25. 18 -msg recd@thirtyeight pence,spam


In [178]:
total = 1114
correct = 0

In [179]:
for row in testing_data.iterrows():
    actual_classifcation = row[1]['Label']
    predicted_classifcation = row[1]['predicted']
    if actual_classifcation == predicted_classifcation:
        correct += 1

In [180]:
accuracy = correct / total

In [181]:
print(accuracy)

0.981149012567325


The accuracy of the Naive Bayes filter is suprisingly good!